# Child Tax Credit

The Child Tax Credit (CTC) is a partially refundable tax credit for filers with children that phases in and out with income.


In [1]:
from policyengine_us.tools.documentation import (
    variable_summary,
    variation_chart,
)

variable_summary("ctc")

Unable to load OpenFisca variables from file "/Users/maxghenis/PolicyEngine/policyengine-us/policyengine_us/variables/contrib/ubi_center/flat_tax.py"


AttributeError: module 'sortedcontainers' has no attribute 'sorteddict'

In [ ]:
variation_chart("ctc")

In [ ]:
from policyengine_us import Simulation
import pandas as pd
import plotly.express as px

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}


def make_ctc(adults, children):
    # Create household structure
    members = ["head"]
    situation = {
        "people": {
            "head": {"age": {"2022": 25}}
        }
    }
    
    if adults == 2:
        members.append("spouse")
        situation["people"]["spouse"] = {"age": {"2022": 25}}
    
    for i in range(children):
        child = f"child{i}"
        members.append(child)
        situation["people"][child] = {"age": {"2022": 6}}
    
    situation["tax_units"] = {"tax_unit": {"members": members}}
    situation["households"] = {
        "household": {
            "members": members,
            "state_name": {"2022": "CA"}  # Default state
        }
    }
    
    # Calculate for different income levels
    results = []
    for income in range(0, 60_000, 100):
        situation["people"]["head"]["employment_income"] = {"2022": income}
        sim = Simulation(situation=situation)
        
        # Calculate CTC
        ctc = sim.calculate("ctc", 2022)[0]
        
        # Calculate marginal rate
        situation_plus = situation.copy()
        situation_plus["people"]["head"]["employment_income"] = {"2022": income + 100}
        sim_plus = Simulation(situation=situation_plus)
        ctc_plus = sim_plus.calculate("ctc", 2022)[0]
        mtr = -(ctc_plus - ctc) / 100
        
        results.append({
            "employment_income": income,
            "ctc": round(ctc),
            "mtr": mtr,
            "adults": adults,
            "children": str(children),
        })
    
    return pd.DataFrame(results)


# Make a table of CTC amounts for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_ctc(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    income_source="Income source",
    ctc="Child tax credit",
    mtr="CTC marginal tax rate",
    children="Children",
    adults="Adults",
)

fig = px.line(
    df,
    "employment_income",
    "ctc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Child tax credit, 2022",
    color_discrete_map=COLOR_MAP,
)

fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

Now compute the net income impact of un-repealing the CTC.


In [ ]:
from policyengine_us import Simulation
import pandas as pd
import plotly.express as px
import copy

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}


def make_net_income(adults, children, year, neutralize_ctc=False):
    # Create household structure
    members = ["head"]
    situation = {
        "people": {
            "head": {"age": {str(year): 25}}
        }
    }
    
    if adults == 2:
        members.append("spouse")
        situation["people"]["spouse"] = {"age": {str(year): 25}}
    
    for i in range(children):
        child = f"child{i}"
        members.append(child)
        situation["people"][child] = {"age": {str(year): 6}}
    
    situation["tax_units"] = {"tax_unit": {"members": members}}
    situation["households"] = {
        "household": {
            "members": members,
            "state_name": {str(year): "CA"}
        }
    }
    
    # Apply reform if needed
    reform = None
    if neutralize_ctc:
        reform = {
            "gov.irs.credits.ctc.amount.max[JOINT].2017-01-01": {"2017-01-01": 0},
            "gov.irs.credits.ctc.amount.max[SINGLE].2017-01-01": {"2017-01-01": 0},
            "gov.irs.credits.ctc.amount.max[SEPARATE].2017-01-01": {"2017-01-01": 0},
            "gov.irs.credits.ctc.amount.max[HEAD_OF_HOUSEHOLD].2017-01-01": {"2017-01-01": 0},
            "gov.irs.credits.ctc.amount.max[WIDOW].2017-01-01": {"2017-01-01": 0},
        }
    
    # Calculate for different income levels
    results = []
    for income in range(0, 200_000, 1_000):
        situation["people"]["head"]["employment_income"] = {str(year): income}
        
        if reform:
            sim = Simulation(situation=situation, reform=reform)
        else:
            sim = Simulation(situation=situation)
        
        income_tax = sim.calculate("income_tax", year)[0]
        
        results.append({
            "employment_income": income,
            "income_tax": income_tax,
        })
    
    return pd.DataFrame(results)


def get_ctc_impact(adults, children, year):
    baseline = make_net_income(adults, children, year, neutralize_ctc=False)
    reform = make_net_income(adults, children, year, neutralize_ctc=True)
    
    return pd.DataFrame({
        "adults": adults,
        "children": str(children),
        "year": year,
        "employment_income": baseline.employment_income,
        "baseline": baseline.income_tax,
        "reform": reform.income_tax,
        "ctc": reform.income_tax - baseline.income_tax,
    })


# Make a table of CTC amounts for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        for year in [2020, 2021, 2022]:
            l.append(get_ctc_impact(adults, children, year))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    income_source="Income source",
    ctc="Child tax credit impact on tax liability",
    mtr="CTC marginal tax rate",
    children="Children",
    adults="Adults",
)

fig = px.line(
    df,
    "employment_income",
    "ctc",
    color="children",
    facet_col="adults",
    animation_frame="year",
    labels=LABELS,
    title="Child tax credit impact by year",
    color_discrete_map=COLOR_MAP,
)

fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()